## Proyecto final
### MongoDB: Pedidos y Ordenes

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
#import pprint
#from bson.objectid import ObjectId
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [22]:
conn_str = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn_str,
                             server_api=ServerApi('1'),
                             serverSelectionTimeoutMS=5000)

# Probando conexión
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

# Borrar base y volver a crear
#client.drop_database('Tienda')

db = client['Tienda']

{'version': '5.0.8', 'gitVersion': 'c87e1c23421bf79614baf500fda6622bd90f674e', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 8, 0], 'openssl': {'running': 'OpenSSL 1.1.1f  31 Mar 2020', 'compiled': 'OpenSSL 1.1.1f  31 Mar 2020'}, 'buildEnvironment': {'distmod': 'ubuntu2004', 'distarch': 'aarch64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-Werror -include mongo/platform/basic.h -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=armv8.2-a -mtune=generic -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -Wa,--nocompress-debug-sections -moutline-atomics', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.5.0', 'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialize

### Colecciones:

* Pedidos
* Órdenes

In [23]:
# Registro de una nueva orden
def nueva_orden(compra:list, fecha:datetime = datetime.now()):
    hora = fecha.strftime('%H:%M:%S')
    fecha = fecha.strftime('%Y-%m-%d')
    total_orden = sum([x[2] for x in compra])
    compra = [{'producto_id': x[0], 'cantidad': x[1], 'subtotal': x[2]} for x in compra]
    orden = {'fecha': fecha ,
             'hora' : hora, 
             'compra': compra,
             'total_orden': total_orden }
    orden_id = db.Ordenes.insert_one(orden).inserted_id
    return orden_id

# Registro de un nuevo pedido
def nuevo_pedido(fecha_entrega:datetime, nombre_cliente:str, telefono:str, compra:list, fecha_apertura:datetime = datetime.now()):
    hora_apertura = fecha_apertura.strftime('%H:%M:%S')
    fecha_apertura = fecha_apertura.strftime('%Y-%m-%d')
    hora_entrega = fecha_entrega.strftime('%H:%M:%S')
    fecha_entrega = fecha_entrega.strftime('%Y-%m-%d')
    total_orden = sum([x[2] for x in compra])
    compra = [{'producto_id': x[0], 'cantidad': x[1], 'subtotal': x[2]} for x in compra] 
    pedido = {'fecha_apertura': fecha_apertura ,
              'hora_apertura' : hora_apertura,
             'fecha_entrega': fecha_entrega,
             'hora_entrega' : hora_entrega,
             'nombre_cliente': nombre_cliente,
             'telefono': telefono,
             'compra': compra,
             'total_orden': total_orden }
    pedido_id = db.Pedidos.insert_one(pedido).inserted_id

    return pedido_id

## Queries:

1. Total de venta de ordenes por día.
4. Consultar pedidos por fecha de entrega.
2. Total de dinero generado por día por anticipo de pedidos.
2. Total de dinero generado por pago de restante de pedidos por día.
5. Consultar total de cada producto a entregar por día

In [24]:
# 1.Total de venta de órdenes por día:
def venta_ordenes_por_dia(fecha:datetime): 
    fecha = fecha.strftime('%Y-%m-%d')
    total = db.Ordenes.aggregate([{ "$match": { "fecha": fecha}},
                                  { "$group": { "_id": "$fecha", "total_dia" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_dia" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]

# 2. Consultar pedidos por fecha de entrega
def consultar_entregas(fecha_entrega:datetime = datetime.today(), format:bool=True):
    fecha_entrega = fecha_entrega.strftime('%Y-%m-%d')
    resultado = db.Pedidos.find({ "fecha_entrega": fecha_entrega}).sort('hora_entrega')
    
    if format:
        df = pd.DataFrame()
        for p in resultado:
            l = []
            for prod in p['compra']:
                l.append('{} : {}'.format(prod['producto_id'], prod['cantidad']))
            columna = []
            columna.append('Hora: {}'.format(p['hora_entrega']))
            columna.append('Tel: {}'.format(p['telefono']))
            total = "${:,}".format(p['total_orden'])
            columna.append('Total: {}'.format(total))
            columna = columna + l
            df = pd.concat([df, pd.DataFrame(columna, columns = [p['nombre_cliente']])], axis=1)
            #df[p['nombre_cliente']] = columna
        df = df.fillna('')
        return df

    pedidos = []
    for p in resultado:
        pedidos.append(p)
    return pedidos


# 3. Total de pagos de anticipos por día
def pago_anticipos_por_dia(fecha_apertura:datetime = datetime.today()):
    fecha_apertura = fecha_apertura.strftime('%Y-%m-%d')
    total = db.Pedidos.aggregate([{ "$match": { "fecha_apertura": fecha_apertura}},
                                   { "$group": { "_id": "$fecha_apertura", "total_anticipos_dia" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_anticipos_dia" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]['total_anticipos_dia']*0.5


# 4. Total de pagos de entregas por día
def pago_entregas_por_dia(fecha_entrega:datetime = datetime.today()):
    fecha_entrega = fecha_entrega.strftime('%Y-%m-%d')
    total = db.Pedidos.aggregate([{ "$match": { "fecha_entrega": fecha_entrega}},
                                   { "$group": { "_id": "fecha_entrega", "total_pago_entregas" : {'$sum': '$total_orden'} }},
                                   { "$project": { "_id":0, "total_pago_entregas" : 1 }}])
    res = []                     
    for t in total:
        res.append(t)
    return res[0]['total_pago_entregas']*0.5


#5. Consultar cantidades de productos a entregar por día
def productos_para_entregas(fecha_entrega:datetime = datetime.today()):
    fecha_entrega = fecha_entrega.strftime('%Y-%m-%d')
    resultado = db.Pedidos.aggregate([{ "$match": { "fecha_entrega": fecha_entrega}},
                                      { "$unwind": "$compra" },
                                      { "$group": { "_id": "$compra.producto_id", "total_piezas" : {'$sum':'$compra.cantidad'} }},
                                      { "$project": { "_id":1, "total_piezas" : 1 }}])
    pedidos = []                     
    for p in resultado:
        pedidos.append(p)
    return pd.DataFrame(pedidos).sort_values('total_piezas', axis=0, ascending=False)


### Testing

In [25]:
# Crear ordenes:
compra_1 = [['jamon', 3, 45], ['piña', 1, 15], ['zarzamora', 2, 40]]
compra_2 = [['jamon', 6, 90], ['piña', 2, 30]]

nueva_orden(compra_1)
nueva_orden(compra_2)

ObjectId('629d3c983c93dc143ada5dd8')

In [26]:
hoy = datetime.now()


print(venta_ordenes_por_dia(hoy))

{'total_dia': 220}


In [27]:
# Crear un pedido:
compra_1 = [['jamon', 6, 90], ['piña', 2, 30], ['zarzamora', 4, 80]]
compra_2 = [['jamon', 6, 90], ['piña', 2, 30]]
compra_3 = [['jamon', 6, 90], ['piña', 2, 30],['zarzamora', 4, 80], ['queso', 10, 200] ]

otro_dia_1 = datetime.strptime('2022-06-10 11:35:00', "%Y-%m-%d %H:%M:%S")
otro_dia_2 = datetime.strptime('2022-06-10 13:00:00', "%Y-%m-%d %H:%M:%S")
otro_dia_3 = datetime.strptime('2022-06-10 12:00:00', "%Y-%m-%d %H:%M:%S")


nuevo_pedido(otro_dia_1,'Marcela Cruz', '93838381005', compra_1)
nuevo_pedido(otro_dia_2,'Yeudiel Lara', '63835581007', compra_2)
nuevo_pedido(otro_dia_3,'Pablo Castillo', '95538371007', compra_3)

ObjectId('629d3c983c93dc143ada5ddb')

In [28]:
consultar_entregas(otro_dia_1)

,Marcela Cruz,Pablo Castillo,Yeudiel Lara
0,Hora: 11:35:00,Hora: 12:00:00,Hora: 13:00:00
1,Tel: 93838381005,Tel: 95538371007,Tel: 63835581007
2,Total: $200,Total: $400,Total: $120
3,jamon : 6,jamon : 6,jamon : 6
4,piña : 2,piña : 2,piña : 2
5,zarzamora : 4,zarzamora : 4,
6,,queso : 10,


In [29]:
pago_anticipos_por_dia(hoy)

360.0

In [30]:
pago_entregas_por_dia(otro_dia_1)

360.0

In [31]:
productos_para_entregas(otro_dia_2)


,_id,total_piezas
0,jamon,18
3,queso,10
2,zarzamora,8
1,piña,6
